 PDF 분할 작업을 위해 `unstructured`를 사용한다`unstructured` 를 위해 다음과 도구의 설치가 필요하다

  - tesseract : 광학 문자 인식(OCR)을 위해 사용
  - poppler : PDF 렌더링 및 처리

  [poppler 설치 방법](https://pdf2image.readthedocs.io/en/latest/installation.html)과 [tesseract 설치 방법](https://tesseract-ocr.github.io/tessdoc/Installation.html)을 참고

# **라이브러리 설치**

In [ ]:
%%capture --no-stderr
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!sudo apt-get install poppler-utils

In [ ]:
%%capture --no-stderr
! pip install -U langchain openai chromadb langchain-experimental langchain_openai nltk pydantic lxml matplotlib chromadb tiktoken
! pip install pillow == 11.1.0
! pip install "unstructured[all-docs]" == 0.17.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# **PDF 데이터 전처리**

In [ ]:
# 파일 경로
fpath = 'pdf 파일이 저장된 디렉토리 경로'
fname = "sample.pdf"

In [ ]:
import nltk

# nltk 필요 데이터 다운로드
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
from unstructured.partition.pdf import partition_pdf
import os

# PDF에서 요소 추출
raw_pdf_elements = partition_pdf(
    filename=os.path.join(fpath, fname),
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    extract_image_block_output_dir=fpath,
)

In [ ]:
# 텍스트, 테이블 추출
tables = []
texts = []

for element in raw_pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
    # 테이블 요소 추가
    tables.append(str(element))
  elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
    # 텍스트 요소 추가
    texts.append(str(element))

In [ ]:
tables[0]
texts[0]

# **텍스트 및 테이블 요약**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 설정
prompt_text = """당신은 표와 텍스트를 요약하여 검색할 수 있도록 돕는 역할을 맡은 어시스턴트입니다.
이 요약은 임베딩되어 원본 텍스트나 표 요소를 검색하는 데 사용될 것입니다.
표 또는 텍스트에 대한 간결한 요약을 제공하여 검색에 최적화된 형태로 만들어 주세요. 표 또는 텍스트: {element} """

prompt = ChatPromptTemplate.from_template(prompt_text)

# 텍스트 요약 체인
model = ChatOpenAI(temperature=0, model="gpt-4")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# 제공된 텍스트에 대해 요약을 할 경우
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# 요약을 원치 않을 경우
# text_summaries = texts

# 제공된 테이블에 적용
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [ ]:
table_summaries[0]
text_summaries[0]

# **base64 인코딩 전달**

In [ ]:
import base64

# -> str = 타입 힌트, 반환값
def encode_image(image_path) -> str:
  # 이미지 base64 인코딩
  # rb = 바이너리 읽기 모드
  with open(image_path, "rb") as image_file:
    # .decode('utf-8') = 바이트를 문자열로 변환
    return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지의 base64 인코딩을 저장하는 리스트
img_base64_list = []

# 이미지를 읽어 bese64 인코딩 후 저장
# os.listdir(fpath) = 해당 경로에 있는 파일 목록 불러오기
for img_file in sorted(os.listdir(fpath)):
  if img_file.endswith('.jpg')
    img_path = os.path.join(fpath, img_file)
    base64_image = encode_image(img_path)
    img_base64_list.append(base64_image)